In [1]:
import sys, os
lib_path = os.path.abspath(os.path.join('..'))
sys.path.append(lib_path)
import tensorflow as tf
import csv
import numpy as np
import tflib as lib
import tflib.ops.linear
import tflib.ops.conv1d
#import language_helpers
import time
from sklearn.utils import shuffle

In [2]:
score_t = []
how_win_t = []
with open('../data/generate_data_ave.txt') as f:
    file = f.readlines()
    for data in file:
        s = data.split()
        score_one = s[0]
        how_win_one = s[1]
        scores = score_one.split('-')
        score1 = round(float(scores[0]),1)
        score2 = round(float(scores[1]),1)
        score_t.append((score1, score2))
        how_win_t.append(how_win_one)

print(len(score_t))

10100


In [3]:
score_t, how_win_t = shuffle(score_t, how_win_t)
t_len = len(score_t)
score = score_t[0:int(t_len*0.8)]
score_test = score_t[int(t_len*0.8):]
how_win = how_win_t[0:int(t_len*0.8)]
how_win_test = how_win_t[int(t_len*0.8):]

In [4]:
"""
team = []
score_t = []
how_win_t = []
sentence = []
with open('../data_set.csv') as f:
    f_csv = csv.reader(f)
    headers = next(f_csv)
    for row in f_csv:
        teams = row[0].split('_')
        team.append((teams[0], teams[1]))
        scores = row[1].split('-')
        score_t.append((int(scores[0]), int(scores[1])))
        how_win_t.append(row[2])
        sentence.append(row[3])
        
t_len = len(score_t)
score = score_t[0:int(t_len*0.8)]
score_test = score_t[int(t_len*0.8):]
how_win = how_win_t[0:int(t_len*0.8)]
how_win_test = how_win_t[int(t_len*0.8):]


"""


"\nteam = []\nscore_t = []\nhow_win_t = []\nsentence = []\nwith open('../data_set.csv') as f:\n    f_csv = csv.reader(f)\n    headers = next(f_csv)\n    for row in f_csv:\n        teams = row[0].split('_')\n        team.append((teams[0], teams[1]))\n        scores = row[1].split('-')\n        score_t.append((int(scores[0]), int(scores[1])))\n        how_win_t.append(row[2])\n        sentence.append(row[3])\n        \nt_len = len(score_t)\nscore = score_t[0:int(t_len*0.8)]\nscore_test = score_t[int(t_len*0.8):]\nhow_win = how_win_t[0:int(t_len*0.8)]\nhow_win_test = how_win_t[int(t_len*0.8):]\n\n\n"

In [5]:
#for i in range(len(score)):
#    print(score[i], how_win[i])

In [8]:
BATCH_SIZE = 64
SEQ_LEN = 2
SCORE_DIM = 20  #generator score dimention
DIM = 128
LAMBDA = 10 # Gradient penalty lambda hyperparameter.
ITERS = 5000
CRITIC_ITERS = 10
NOIZE_DIM = 10

In [9]:
inv_charmap = []
inv_charmap.append('unk')
charmap = {'unk':0}
for word in how_win:
    if word not in inv_charmap:
        charmap[word] = len(inv_charmap)
        inv_charmap.append(word)

In [10]:
inv_charmap

['unk', '击败', '大败', '不敌', '险胜', '惜败', '战胜', '大胜', '掀翻', '痛宰', '轻取']

In [11]:
def get_next_batch(batch_size, _score, _how_win, index):
    length = len(score)
    start_point = index * batch_size
    batch_gen_score = []
    batch_disc_score = []
    batch_how_win = []
    for i in range(batch_size):
        current_point = start_point + i
        if current_point >= length:
            current_point -= length
        
        one_score = _score[current_point]
        gen_score = [one_score[0]/200.0 for i in range(int(SCORE_DIM/2))] + [one_score[1]/200.0 for i in range(int(SCORE_DIM/2))]
        batch_gen_score.append(gen_score)
        
        disc_score = [[one_score[0]/200.0 for i in range(len(charmap))]] + [[one_score[1]/200.0 for i in range(len(charmap))]]
        batch_disc_score.append(disc_score)
        
        batch_how_win.append([charmap[_how_win[current_point]],0])

    
    return batch_gen_score, batch_disc_score, batch_how_win


In [12]:
def softmax(logits):
    return tf.reshape(
        tf.nn.softmax(
            tf.reshape(logits, [-1, len(charmap)])
        ),
        tf.shape(logits)
    )
def make_noise(shape):
    return tf.random_normal(shape)

def ResBlock(name, inputs): 
    output = inputs
    output = tf.nn.relu(output)
    output = lib.ops.conv1d.Conv1D(name+'.1', DIM, DIM, 5, output)
    output = tf.nn.relu(output)
    #output = tf.expand_dims(output, 2)
    #print(output)
    output = lib.ops.conv1d.Conv1D(name+'.2', DIM, DIM, 5, output)
    #output = tf.expand_dims(output, 2)
    return inputs + (0.3*output)

In [13]:
def Generator(n_samples, scores, prev_outputs=None):
    output = make_noise(shape=[n_samples, NOIZE_DIM])
    output = tf.concat([scores, output], 1)
    output = lib.ops.linear.Linear('Generator.Input', SCORE_DIM+NOIZE_DIM, SEQ_LEN*DIM, output)
    output = tf.reshape(output, [-1, DIM, SEQ_LEN])
    output = ResBlock('Generator.1', output)
    output = ResBlock('Generator.2', output)
    output = ResBlock('Generator.3', output)
    output = ResBlock('Generator.4', output)
    output = ResBlock('Generator.5', output)
    output = lib.ops.conv1d.Conv1D('Generator.Output', DIM, len(charmap), 1, output)
    
    #output = tf.expand_dims(output, 2)
    output = tf.transpose(output, [0, 2, 1])
    output = softmax(output)
    return output

#test
"""
score = tf.placeholder(tf.float32, shape=[BATCH_SIZE, 20])
Generator(BATCH_SIZE, score)
"""

'\nscore = tf.placeholder(tf.float32, shape=[BATCH_SIZE, 20])\nGenerator(BATCH_SIZE, score)\n'

In [14]:
def Discriminator(inputs, scores):
    output = tf.concat([inputs, scores], 1)
    output = tf.transpose(output, [0,2,1])
    output = lib.ops.conv1d.Conv1D('Discriminator.Input', len(charmap), DIM, 1, output)
    output = ResBlock('Discriminator.1', output)
    output = ResBlock('Discriminator.2', output)
    output = ResBlock('Discriminator.3', output)
    output = ResBlock('Discriminator.4', output)
    output = ResBlock('Discriminator.5', output)
    output = tf.reshape(output, [-1, (2+SEQ_LEN)*DIM])
    output = lib.ops.linear.Linear('Discriminator.Output', (2+SEQ_LEN)*DIM, 1, output)
    return output

#test
"""
real_inputs_discrete = tf.placeholder(tf.int32, shape=[BATCH_SIZE, SEQ_LEN])
real_inputs = tf.one_hot(real_inputs_discrete, len(charmap))
score = tf.placeholder(tf.float32, shape=[BATCH_SIZE, 2, len(charmap)])
Discriminator(real_inputs, score) 
"""

'\nreal_inputs_discrete = tf.placeholder(tf.int32, shape=[BATCH_SIZE, SEQ_LEN])\nreal_inputs = tf.one_hot(real_inputs_discrete, len(charmap))\nscore = tf.placeholder(tf.float32, shape=[BATCH_SIZE, 2, len(charmap)])\nDiscriminator(real_inputs, score) \n'

In [15]:
score_for_gen = tf.placeholder(tf.float32, shape=[None, SCORE_DIM])
score_for_disc = tf.placeholder(tf.float32, shape=[None, 2, len(charmap)])
real_inputs_discrete = tf.placeholder(tf.int32, shape=[None, SEQ_LEN])
real_inputs = tf.one_hot(real_inputs_discrete, len(charmap))
fake_inputs = Generator(BATCH_SIZE, score_for_gen)
fake_inputs_discrete = tf.argmax(fake_inputs, fake_inputs.get_shape().ndims-1)

sample_size = tf.placeholder(tf.int32, shape=[])
sample_inputs = Generator(sample_size, score_for_gen)



disc_real = Discriminator(real_inputs, score_for_disc) 
disc_fake = Discriminator(fake_inputs, score_for_disc)

disc_cost = tf.reduce_mean(disc_fake) - tf.reduce_mean(disc_real)
gen_cost = -tf.reduce_mean(disc_fake)

In [16]:
# WGAN lipschitz-penalty
alpha = tf.random_uniform(
    shape=[BATCH_SIZE,1,1], 
    minval=0.,
    maxval=1.
)
differences = fake_inputs - real_inputs
interpolates = real_inputs + (alpha*differences)
gradients = tf.gradients(Discriminator(interpolates, score_for_disc), [interpolates])[0]
slopes = tf.sqrt(tf.reduce_sum(tf.square(gradients), reduction_indices=[1,2]))
gradient_penalty = tf.reduce_mean((slopes-1.)**2)
disc_cost += LAMBDA*gradient_penalty

gen_params = lib.params_with_name('Generator')
disc_params = lib.params_with_name('Discriminator')

gen_train_op = tf.train.AdamOptimizer(learning_rate=1e-4, beta1=0.5, beta2=0.9).minimize(gen_cost, var_list=gen_params)
disc_train_op = tf.train.AdamOptimizer(learning_rate=1e-4, beta1=0.5, beta2=0.9).minimize(disc_cost, var_list=disc_params)

下面的训练结果格式是：

（比分1，比分2）（预测结果，实际结果）

In [17]:
with tf.Session() as session:
    
    session.run(tf.global_variables_initializer())
    sc_gen = None 
    sc_disc = None
    _data = None
    def generate_samples():
        sc_gen, sc_disc, _data = get_next_batch(len(score_test), score_test, how_win_test, 0)
        samples = session.run(sample_inputs, feed_dict={sample_size: len(sc_gen), score_for_gen:sc_gen, score_for_disc:sc_disc})
        samples = np.argmax(samples, axis=2)
        decoded_samples = []
        for i in range(len(samples)):
            decoded_samples.append((inv_charmap[samples[i][0]], inv_charmap[_data[i][0]]))
        return decoded_samples
    
    batch_index = 0
    for iteration in range(ITERS):
        start_time = time.time()
        
        # Train generator
        if iteration > 0:
            _ = session.run(gen_train_op, feed_dict={score_for_gen:sc_gen, score_for_disc:sc_disc})
            
        
         # Train critic
        for i in range(CRITIC_ITERS):
            if batch_index * BATCH_SIZE > len(score):
                batch_index = 0
            sc_gen, sc_disc, _data= get_next_batch(BATCH_SIZE, score, how_win, batch_index)
            batch_index += 1
            _disc_cost, _gen_cost, _gradient_penalty,  _ = session.run(
                [disc_cost, gen_cost, gradient_penalty, disc_train_op],
                feed_dict={real_inputs_discrete:_data, score_for_gen:sc_gen, score_for_disc: sc_disc}
            )
        
        #lib.plot.plot('time', time.time() - start_time)
        #lib.plot.plot('train disc cost', _disc_cost)
        
        #if iteration % 100 == 99:
            #lib.plot.flush()
        
        #lib.plot.tick()
        if iteration % 10 == 0:
            print(iteration, _disc_cost, _gen_cost, _gradient_penalty)
        
        if iteration % 100 == 0:
            sample = generate_samples()
            for i in range(len(sample)):
                print(score_test[i], sample[i])
        
        if iteration % 500 == 0:
            saver = tf.train.Saver()
            saver.save(session, 'trained_model',global_step=iteration)
    

0 -0.399711 0.654731 0.00192651
(101.0, 97.0) ('击败', '险胜')
(111.0, 98.0) ('轻取', '击败')
(109.0, 87.0) ('击败', '轻取')
(113.0, 80.0) ('掀翻', '痛宰')
(109.0, 119.0) ('轻取', '不敌')
(109.0, 127.0) ('击败', '大败')
(118.0, 95.0) ('轻取', '轻取')
(103.0, 121.0) ('击败', '大败')
(124.0, 119.0) ('击败', '险胜')
(120.0, 98.0) ('不敌', '轻取')
(114.0, 91.0) ('击败', '轻取')
(123.0, 141.0) ('大败', '大败')
(114.0, 133.0) ('不敌', '大败')
(113.0, 134.0) ('轻取', '大败')
(116.0, 82.0) ('轻取', '痛宰')
(118.0, 127.0) ('击败', '不敌')
(112.0, 106.0) ('击败', '险胜')
(112.0, 133.0) ('轻取', '大败')
(102.0, 107.0) ('险胜', '惜败')
(107.0, 117.0) ('不敌', '不敌')
(103.0, 80.0) ('击败', '轻取')
(124.0, 113.0) ('轻取', '击败')
(106.0, 92.0) ('击败', '击败')
(129.0, 123.0) ('掀翻', '险胜')
(127.0, 113.0) ('击败', '击败')
(116.0, 105.0) ('大败', '战胜')
(120.0, 106.0) ('击败', '击败')
(114.0, 79.0) ('击败', '痛宰')
(117.0, 94.0) ('击败', '轻取')
(100.0, 65.0) ('轻取', '痛宰')
(112.0, 109.0) ('轻取', '掀翻')
(111.0, 108.0) ('击败', '掀翻')
(114.0, 82.0) ('击败', '痛宰')
(129.0, 95.0) ('击败', '痛宰')
(128.0, 132.0) ('轻取', '惜败')
(11

(103.0, 79.0) ('轻取', '轻取')
(103.0, 80.0) ('击败', '轻取')
(121.0, 131.0) ('击败', '不敌')
(105.0, 114.0) ('大败', '不敌')
(102.0, 105.0) ('击败', '惜败')
(123.0, 119.0) ('轻取', '掀翻')
(126.0, 100.0) ('击败', '大胜')
(111.0, 86.0) ('轻取', '大胜')
(128.0, 139.0) ('痛宰', '不敌')
(112.0, 100.0) ('击败', '战胜')
(104.0, 78.0) ('击败', '大胜')
(113.0, 109.0) ('大败', '险胜')
(114.0, 133.0) ('轻取', '大败')
(116.0, 81.0) ('大败', '痛宰')
(108.0, 96.0) ('轻取', '击败')
(101.0, 68.0) ('大胜', '痛宰')
(115.0, 123.0) ('击败', '不敌')
(122.0, 119.0) ('击败', '险胜')
(102.0, 92.0) ('大败', '战胜')
(101.0, 112.0) ('击败', '不敌')
(126.0, 144.0) ('击败', '大败')
(128.0, 133.0) ('不敌', '惜败')
(118.0, 121.0) ('险胜', '惜败')
(128.0, 137.0) ('轻取', '不敌')
(129.0, 131.0) ('大败', '惜败')
(102.0, 106.0) ('大败', '惜败')
(116.0, 110.0) ('险胜', '险胜')
(100.0, 87.0) ('击败', '战胜')
(103.0, 98.0) ('击败', '险胜')
(109.0, 95.0) ('击败', '击败')
(105.0, 93.0) ('轻取', '战胜')
(122.0, 116.0) ('轻取', '掀翻')
(114.0, 118.0) ('击败', '惜败')
(115.0, 135.0) ('痛宰', '大败')
(111.0, 116.0) ('击败', '惜败')
(123.0, 102.0) ('险胜', '轻取')
(100

(109.0, 97.0) ('击败', '战胜')
(103.0, 71.0) ('掀翻', '痛宰')
(116.0, 137.0) ('大败', '大败')
(110.0, 107.0) ('击败', '掀翻')
(126.0, 99.0) ('大败', '大胜')
(101.0, 96.0) ('轻取', '险胜')
(124.0, 120.0) ('轻取', '掀翻')
(104.0, 77.0) ('击败', '大胜')
(123.0, 120.0) ('轻取', '险胜')
(117.0, 137.0) ('轻取', '大败')
(121.0, 107.0) ('轻取', '击败')
(123.0, 113.0) ('轻取', '战胜')
(115.0, 104.0) ('大败', '击败')
(113.0, 134.0) ('击败', '大败')
(119.0, 87.0) ('击败', '痛宰')
(122.0, 110.0) ('大败', '击败')
(118.0, 85.0) ('击败', '痛宰')
(108.0, 116.0) ('轻取', '不敌')
(115.0, 80.0) ('险胜', '痛宰')
(119.0, 98.0) ('击败', '轻取')
(129.0, 148.0) ('险胜', '大败')
(124.0, 128.0) ('大败', '惜败')
(118.0, 112.0) ('不敌', '掀翻')
(127.0, 102.0) ('掀翻', '大胜')
(126.0, 92.0) ('大败', '痛宰')
(126.0, 120.0) ('大败', '险胜')
(117.0, 136.0) ('大败', '大败')
(110.0, 129.0) ('击败', '大败')
(112.0, 80.0) ('大败', '痛宰')
(105.0, 100.0) ('击败', '掀翻')
(109.0, 77.0) ('轻取', '痛宰')
(127.0, 116.0) ('轻取', '战胜')
(110.0, 82.0) ('击败', '大胜')
(121.0, 107.0) ('击败', '击败')
(104.0, 124.0) ('击败', '大败')
(129.0, 117.0) ('轻取', '战胜')
(116.

10 -1.11394 0.702481 0.00400937
20 -0.871094 1.49056 0.0031182
30 -0.752219 2.3039 0.0026793
40 -0.759267 2.7783 0.00318337
50 -0.745481 3.36932 0.00377269
60 -0.754883 3.60485 0.00433288
70 -0.656875 4.09909 0.00292277
80 -0.645122 4.41713 0.00376528
90 -0.700296 4.72155 0.00634234
100 -0.599897 4.89364 0.00622587
(101.0, 97.0) ('大胜', '险胜')
(111.0, 98.0) ('大胜', '击败')
(109.0, 87.0) ('惜败', '轻取')
(113.0, 80.0) ('掀翻', '痛宰')
(109.0, 119.0) ('掀翻', '不敌')
(109.0, 127.0) ('掀翻', '大败')
(118.0, 95.0) ('惜败', '轻取')
(103.0, 121.0) ('不敌', '大败')
(124.0, 119.0) ('不敌', '险胜')
(120.0, 98.0) ('险胜', '轻取')
(114.0, 91.0) ('击败', '轻取')
(123.0, 141.0) ('击败', '大败')
(114.0, 133.0) ('惜败', '大败')
(113.0, 134.0) ('不敌', '大败')
(116.0, 82.0) ('击败', '痛宰')
(118.0, 127.0) ('大胜', '不敌')
(112.0, 106.0) ('险胜', '险胜')
(112.0, 133.0) ('掀翻', '大败')
(102.0, 107.0) ('不敌', '惜败')
(107.0, 117.0) ('掀翻', '不敌')
(103.0, 80.0) ('掀翻', '轻取')
(124.0, 113.0) ('大胜', '击败')
(106.0, 92.0) ('大胜', '击败')
(129.0, 123.0) ('掀翻', '险胜')
(127.0, 113.0) ('不敌',

(122.0, 127.0) ('不敌', '惜败')
(127.0, 124.0) ('不敌', '掀翻')
(124.0, 132.0) ('不敌', '不敌')
(102.0, 98.0) ('击败', '险胜')
(104.0, 112.0) ('掀翻', '不敌')
(117.0, 92.0) ('击败', '大胜')
(125.0, 121.0) ('大胜', '掀翻')
(119.0, 86.0) ('大胜', '痛宰')
(122.0, 130.0) ('大胜', '不敌')
(117.0, 82.0) ('掀翻', '痛宰')
(121.0, 142.0) ('不敌', '大败')
(105.0, 113.0) ('掀翻', '不敌')
(104.0, 91.0) ('大胜', '击败')
(128.0, 124.0) ('击败', '掀翻')
(103.0, 68.0) ('不敌', '痛宰')
(102.0, 110.0) ('击败', '不敌')
(118.0, 91.0) ('击败', '大胜')
(121.0, 110.0) ('惜败', '战胜')
(121.0, 109.0) ('掀翻', '战胜')
(129.0, 126.0) ('大胜', '掀翻')
(122.0, 124.0) ('不敌', '惜败')
(109.0, 111.0) ('险胜', '惜败')
(106.0, 81.0) ('大胜', '大胜')
(109.0, 96.0) ('掀翻', '击败')
(128.0, 124.0) ('惜败', '险胜')
(126.0, 91.0) ('掀翻', '痛宰')
(115.0, 133.0) ('击败', '大败')
(123.0, 125.0) ('险胜', '惜败')
(110.0, 120.0) ('不敌', '不敌')
(101.0, 119.0) ('险胜', '大败')
(112.0, 80.0) ('击败', '痛宰')
(126.0, 128.0) ('惜败', '惜败')
(109.0, 98.0) ('惜败', '击败')
(116.0, 112.0) ('险胜', '掀翻')
(127.0, 116.0) ('不敌', '击败')
(117.0, 91.0) ('不敌', '大胜')
(119.

...

(116.0, 111.0) ('险胜', '险胜')
(103.0, 76.0) ('大胜', '大胜')
(126.0, 134.0) ('不敌', '不敌')
(127.0, 136.0) ('不敌', '不敌')
(127.0, 123.0) ('险胜', '掀翻')
(112.0, 87.0) ('大胜', '大胜')
(104.0, 81.0) ('大胜', '轻取')
(127.0, 115.0) ('击败', '击败')
(123.0, 120.0) ('掀翻', '险胜')
(106.0, 72.0) ('痛宰', '痛宰')
(115.0, 102.0) ('击败', '战胜')
(101.0, 97.0) ('险胜', '险胜')
(107.0, 109.0) ('惜败', '惜败')
(121.0, 139.0) ('大败', '大败')
(117.0, 89.0) ('大胜', '大胜')
(120.0, 122.0) ('惜败', '惜败')
(109.0, 95.0) ('战胜', '击败')
(100.0, 109.0) ('不敌', '不敌')
(101.0, 96.0) ('掀翻', '掀翻')
(115.0, 135.0) ('大败', '大败')
(110.0, 97.0) ('战胜', '击败')
(120.0, 94.0) ('大胜', '大胜')
(115.0, 91.0) ('轻取', '轻取')
(122.0, 125.0) ('惜败', '惜败')
(101.0, 95.0) ('击败', '险胜')
(106.0, 94.0) ('击败', '击败')
(119.0, 115.0) ('险胜', '险胜')
(113.0, 88.0) ('大胜', '大胜')
(129.0, 116.0) ('战胜', '战胜')
(102.0, 111.0) ('不敌', '不敌')
(128.0, 130.0) ('惜败', '惜败')
(100.0, 94.0) ('战胜', '险胜')
(117.0, 113.0) ('险胜', '险胜')
(120.0, 106.0) ('战胜', '击败')
(116.0, 104.0) ('战胜', '战胜')
(112.0, 109.0) ('险胜', '险胜')
(111.0,

(100.0, 95.0) ('掀翻', '险胜')
(105.0, 82.0) ('大胜', '轻取')
(122.0, 116.0) ('险胜', '险胜')
(127.0, 116.0) ('战胜', '战胜')
(108.0, 104.0) ('掀翻', '掀翻')
(109.0, 105.0) ('险胜', '掀翻')
(120.0, 116.0) ('险胜', '掀翻')
(108.0, 118.0) ('不敌', '不敌')
(129.0, 124.0) ('险胜', '险胜')
(129.0, 105.0) ('轻取', '轻取')
(100.0, 105.0) ('惜败', '惜败')
(101.0, 77.0) ('大胜', '轻取')
(121.0, 139.0) ('大败', '大败')
(113.0, 103.0) ('击败', '战胜')
(123.0, 132.0) ('不敌', '不敌')
(121.0, 110.0) ('战胜', '战胜')
(129.0, 140.0) ('不敌', '不敌')
(116.0, 112.0) ('险胜', '掀翻')
(124.0, 121.0) ('掀翻', '险胜')
(109.0, 112.0) ('惜败', '惜败')
(106.0, 100.0) ('掀翻', '险胜')
(101.0, 90.0) ('击败', '击败')
(112.0, 106.0) ('掀翻', '险胜')
(122.0, 140.0) ('大败', '大败')
(125.0, 93.0) ('痛宰', '痛宰')
(110.0, 106.0) ('险胜', '险胜')
(128.0, 105.0) ('轻取', '轻取')
(109.0, 111.0) ('惜败', '惜败')
(124.0, 126.0) ('惜败', '惜败')
(119.0, 138.0) ('大败', '大败')
(115.0, 125.0) ('不敌', '不敌')
(104.0, 93.0) ('击败', '战胜')
(117.0, 128.0) ('不敌', '不敌')
(115.0, 83.0) ('大胜', '痛宰')
(105.0, 101.0) ('险胜', '掀翻')
(106.0, 94.0) ('战胜', '战胜')


(124.0, 97.0) ('大胜', '大胜')
(108.0, 82.0) ('大胜', '大胜')
(110.0, 98.0) ('击败', '战胜')
(117.0, 95.0) ('轻取', '轻取')
(129.0, 119.0) ('战胜', '战胜')
(128.0, 122.0) ('掀翻', '掀翻')
(124.0, 92.0) ('痛宰', '痛宰')
(128.0, 146.0) ('大败', '大败')
(126.0, 130.0) ('惜败', '惜败')
(105.0, 101.0) ('险胜', '掀翻')
(109.0, 127.0) ('大败', '大败')
(112.0, 86.0) ('大胜', '大胜')
(127.0, 92.0) ('痛宰', '痛宰')
(117.0, 105.0) ('战胜', '击败')
(105.0, 113.0) ('不敌', '不敌')
(128.0, 122.0) ('险胜', '险胜')
(115.0, 110.0) ('掀翻', '掀翻')
(127.0, 130.0) ('惜败', '惜败')
(107.0, 95.0) ('击败', '战胜')
(101.0, 90.0) ('战胜', '战胜')
(123.0, 112.0) ('战胜', '战胜')
(110.0, 120.0) ('不敌', '不敌')
(129.0, 138.0) ('不敌', '不敌')
(112.0, 85.0) ('大胜', '大胜')
(116.0, 111.0) ('险胜', '掀翻')
(100.0, 95.0) ('掀翻', '掀翻')
(118.0, 94.0) ('大胜', '轻取')
(104.0, 92.0) ('战胜', '战胜')
(114.0, 119.0) ('惜败', '惜败')
(142.0, 143.0) ('惜败', '惜败')
(102.0, 89.0) ('战胜', '战胜')
(113.0, 103.0) ('掀翻', '战胜')
(125.0, 91.0) ('痛宰', '痛宰')
(129.0, 140.0) ('不敌', '不敌')
(124.0, 119.0) ('险胜', '险胜')
(126.0, 131.0) ('惜败', '惜败')
(126.0,

3310 -0.139126 -0.660876 0.000619225
3320 -0.358891 -0.199833 0.00210873
3330 -0.140647 -0.474741 0.000768244
3340 -0.165281 -0.527076 0.00117699


KeyboardInterrupt: 

In [18]:
sess=tf.Session()   
sess.run(tf.global_variables_initializer())
saver = tf.train.import_meta_graph('trained_model-3000.meta')
saver.restore(sess,tf.train.latest_checkpoint('./'))


In [19]:
def generate_samples():
        sc_gen, sc_disc, _data = get_next_batch(len(score_test), score_test, how_win_test, 0)
        samples = sess.run(sample_inputs, feed_dict={sample_size: len(sc_gen), score_for_gen:sc_gen, score_for_disc:sc_disc})
        samples = np.argmax(samples, axis=2)
        decoded_samples = []
        for i in range(len(samples)):
            decoded_samples.append((inv_charmap[samples[i][0]], inv_charmap[_data[i][0]]))
        return decoded_samples

下面两个情况说明模型学习到了不同比分下的单词，还是会出现本应“大胜”但是结果是“险胜”情况可能是因为训练次数不够，用的是训练了1500次的结果。

比分是负的情况在数据集里面只有4组，所以可能没学习出来。

下面的结果只出现了4种情况也应该是数据集分布不均匀，这四个词最多，所以其他词出现的概率很低。

In [46]:
#test one 
test_1_score = [(120, 60)]
#whatever
how_win_1 = ['大胜']
one_dict = {}
for i in range(100):
    sc_gen, sc_disc, _data = get_next_batch(len(test_1_score), test_1_score, how_win_1, 0)
    sample = sess.run(sample_inputs, feed_dict={sample_size: len(sc_gen), score_for_gen:sc_gen, score_for_disc:sc_disc})
    sample2 = np.argmax(sample, axis=2)
    hw = inv_charmap[sample2[0][0]]
    one_dict[hw] = one_dict[hw] + 1 if hw in one_dict else 1
"""
print(sc_disc[0][0][0], sc_disc[0][1][0])
print(sample[0][0])
print(sample2[0][0])
print(inv_charmap[sample2[0][0]])
inv_charmap
"""
one_dict

{'痛宰': 100}

In [45]:
#test one 
test_1_score = [(95, 100)]
#whatever
how_win_1 = ['惜败']
one_dict = {}
for i in range(100):
    sc_gen, sc_disc, _data = get_next_batch(len(test_1_score), test_1_score, how_win_1, 0)
    sample = sess.run(sample_inputs, feed_dict={sample_size: len(sc_gen), score_for_gen:sc_gen, score_for_disc:sc_disc})
    sample2 = np.argmax(sample, axis=2)
    hw = inv_charmap[sample2[0][0]]
    one_dict[hw] = one_dict[hw] + 1 if hw in one_dict else 1
"""
print(sc_disc[0][0][0], sc_disc[0][1][0])
print(sample[0][0])
print(sample2[0][0])
print(inv_charmap[sample2[0][0]])
inv_charmap
"""
one_dict

{'不敌': 13, '惜败': 87}